In [51]:
import os
import openai
import dotenv
import pyodbc 
import json

dotenv.load_dotenv()

True

In [52]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

In [53]:
if not use_azure_active_directory:
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2024-02-15-preview"
    )

In [54]:
deployment = "gpt-4" # Fill in the deployment name from the portal here

In [55]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "get_customers",
                "description": "Get the customers for a given city",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The city, e.g. New York",
                        }
                    },
                    "required": ["city"],
                },
            },
        }
]

In [56]:
messages = [
    {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {"role": "user", "content": "Show me customers list for New York city."}
]

chat_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=functions
)
print(chat_completion)





ChatCompletion(id='chatcmpl-9HZ2I6dGyVw0odMcnfZw8Ljsgheb3', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xrpyLJTDoyVDvZObZXcMj56c', function=Function(arguments='{\n  "city": "New York"\n}', name='get_customers'), type='function')]), content_filter_results={})], created=1713973282, model='gpt-4', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=89, total_tokens=106), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])


In [57]:

def get_customers(request):
    """
    This function is for illustrative purposes.
    The city should be used to determine customer
    instead of returning a hardcoded response.
    """
    city = request.get("city")

    cnxn = pyodbc.connect('Driver={ODBC Driver 18 for SQL Server};Server=tcp:' + os.getenv("SQL_DB_URL") + ',1433;Database=' + os.getenv("SQL_DB_NAME") + ';Uid=' + os.getenv("SQL_DB_USER") + ';Pwd=' + os.getenv("SQL_DB_PASSWORD") + ';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
    cursor = cnxn.cursor()	

    sql = 'call SelectAllCustomersParams(?)'
    values = (city)

    sql = """\
    SET NOCOUNT ON;
    DECLARE @RC int;
    EXEC @RC = [bikestore].[dbo].[SelectAllCustomersParams] ?;
    SELECT @RC AS rc;
    """

    cursor.execute(sql, (values))

    rows = []
    row = cursor.fetchone() 
    while row:
        row = cursor.fetchone()
        rows.append(row)

    print(str(rows))
    return str(rows)

function_call = chat_completion.choices[0].message.tool_calls[0].function
print(function_call.name)
print(function_call.arguments)

if function_call.name == "get_customers":
    response = get_customers(json.loads(function_call.arguments))
    print(response)

get_customers
{
  "city": "New York"
}
[(178, 'Genoveva', 'Tyler', '(212) 152-6381', 'genoveva.tyler@gmail.com', '8121 Windfall Ave. ', 'New York', 'NY', '10002'), (327, 'Sharie', 'Alvarez', '(212) 211-7621', 'sharie.alvarez@msn.com', '987 West Leatherwood Dr. ', 'New York', 'NY', '10002'), (411, 'Octavia', 'Case', '(212) 171-1335', 'octavia.case@aol.com', '40 Charles Road ', 'New York', 'NY', '10002'), (854, 'Phylis', 'Adkins', '(212) 325-9145', 'phylis.adkins@msn.com', '7781 James Ave. ', 'New York', 'NY', '10002'), (927, 'Guillermo', 'Hart', '(212) 652-7198', 'guillermo.hart@hotmail.com', '81 Indian Summer Drive ', 'New York', 'NY', '10002'), (1016, 'Shenna', 'Benton', '(212) 578-2912', 'shenna.benton@msn.com', '57 Shadow Brook Road ', 'New York', 'NY', '10002'), None]
[(178, 'Genoveva', 'Tyler', '(212) 152-6381', 'genoveva.tyler@gmail.com', '8121 Windfall Ave. ', 'New York', 'NY', '10002'), (327, 'Sharie', 'Alvarez', '(212) 211-7621', 'sharie.alvarez@msn.com', '987 West Leatherwood

In [58]:
"""
messages.append(
    {
        "role": "function",
        "name": "get_customers",
        "content": json.dumps(response)
    }
)

function_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=functions,
)

print(function_completion.choices[0].message.content.strip())
"""

'\nmessages.append(\n    {\n        "role": "function",\n        "name": "get_customers",\n        "content": json.dumps(response)\n    }\n)\n\nfunction_completion = client.chat.completions.create(\n    model=deployment,\n    messages=messages,\n    tools=functions,\n)\n\nprint(function_completion.choices[0].message.content.strip())\n'